# Homework 2

## Question 1

Write a function that returns the name of a company for a given ticker symbol. You should scrape the data from Yahoo! Finance. [Here's an example page](https://finance.yahoo.com/quote/WMT/profile) using Walmart.

Start by making your function modify the URL so it works for other ticker symbols. Then, look at the HTML source to find the pattern that you must search for to extract the company name.

I've started the function for you here.

In [63]:
def get_name(ticker):
    import requests, re
    
    url = f'https://finance.yahoo.com/quote/{ticker}/profile'
    
    # do the download
    req = requests.get(url)
    html = req.text
    html = html.replace('&#x27;',"'")
    
    try:
        # use a regular expression to find the name
        name = re.search(r'<title>(.+?) \(', html)
        return name
    except:
        # if the regular expression fails, do nothing
        pass

<re.Match object; span=(235, 274), match='<title>Archer-Daniels-Midland Company ('>

Test your function using these tickers by looping over the ticker symbols `ADM`, `AMZN`, `MMM`, `M`, and `WMT` and calling it for each one. The output should look like this:

<img src="https://kelley.iu.edu/nstoffma/da/hw2_q1.png">

In [65]:
tickers = ['ADM', 'AMZN', 'MMM', 'M','WMT']
for ticker in tickers:
    print(ticker, get_name(ticker).group(1))

ADM Archer-Daniels-Midland Company
AMZN Amazon.com, Inc.
MMM 3M Company
M Macy's Inc
WMT Walmart Inc.


## Question 2
Write a function that uses the same web page to find the address of the company's headquarters.

For example, your function should work as follows:

    In  []: get_address('TSLA')
    Out []: '3500 Deer Creek Road, Palo Alto, CA, 94304'
    
Be sure to test your function with some other stock tickers.

Hint: You'll probably want to capture different parts of the address (street address, city, state, zip) separately and then join them together into a string before returning a value. 

In [8]:
def get_address(ticker):
    import requests, re
    
    url='https://finance.yahoo.com/quote/{}/profile'.format(str(ticker))
    
    # do the download
    req = requests.get(url)
    html = req.text
    try:
        # use a regular expression to find the name
        address = re.search('(?<=<!-- react-text: 9 -->).+?(?=<br data-reactid="12"/>)',html).group()
        new = address.replace('<!-- /react-text -->','')
        new2 = new.replace('<br data-reactid="10"/><!-- react-text: 11 -->','')
        return new2
    except:
        # if the regular expression fails, do nothing
        pass

get_address('TSLA')

'3500 Deer Creek RoadPalo Alto, CA 94304'

## Question 3

Now I want you to write a *geolocating* function that takes a ticker symbol and returns the latitude and longitude of the firm's headquarters. To do this, we will use Mapquest's API (<b>A</b>pplication <b>P</b>rogramming <b>I</b>nterface), which is a web-based system for providing raw data rather than HTML pages.

Before accessing the API you need to create an account which generates an API Key. The account is free and allows you to make a limited number of queries. To sign up, go [here](https://developer.mapquest.com/user/register) and choose Sign Up. You are assigned a key when you register. It will look something like this: 

<pre>zAWt8Aye5YvhXupZMzctaAfkSkEnacCn</pre>

Once you have created your key, you can query the API using a URL like this:

http://www.mapquestapi.com/geocoding/v1/address?key=API_KEY&location=1309+E+10th+Street%2C+Bloomington%2C+IN%2C+47401

(This link will *not* work until you replace `API_KEY` with your personal key.)

Notice that URLs use special formatting to deal with spaces and other punctuation.
An easy way to request a web site with parameters like this is as follows:

In [66]:
API_KEY = 'HLnISS3aFXpTqR8YBGAhU7e10KcMXNrA'

import requests

url = 'http://www.mapquestapi.com/geocoding/v1/address'
params = {'key': API_KEY, 'location': '1309 E 10th Street, Bloomington, IN, 47401'}

requests.get(url, params=params).text

'{"info":{"statuscode":0,"copyright":{"text":"\\u00A9 2020 MapQuest, Inc.","imageUrl":"http://api.mqcdn.com/res/mqlogo.gif","imageAltText":"\\u00A9 2020 MapQuest, Inc."},"messages":[]},"options":{"maxResults":-1,"thumbMaps":true,"ignoreLatLngInput":false},"results":[{"providedLocation":{"location":"1309 E 10th Street, Bloomington, IN, 47401"},"locations":[{"street":"1309 E 10th St","adminArea6":"","adminArea6Type":"Neighborhood","adminArea5":"Bloomington","adminArea5Type":"City","adminArea4":"Monroe","adminArea4Type":"County","adminArea3":"IN","adminArea3Type":"State","adminArea1":"US","adminArea1Type":"Country","postalCode":"47405-1701","geocodeQualityCode":"L1AAB","geocodeQuality":"ADDRESS","dragPoint":false,"sideOfStreet":"L","linkId":"rnr3440297|i21503866","unknownInput":"","type":"s","latLng":{"lat":39.171598,"lng":-86.517943},"displayLatLng":{"lat":39.171798,"lng":-86.517941},"mapUrl":"http://www.mapquestapi.com/staticmap/v5/map?key=HLnISS3aFXpTqR8YBGAhU7e10KcMXNrA&type=map&size=

The response from the server is formatted using JSON. But you can simply treat it like any other string and use a regular expression to find the pattern you're looking for.

As an example, your code should return a tuple *exactly* like this:

    In  []: geolocate('TSLA')
    Out []: (37.394738, -122.149112, '3500 Deer Creek Road, Palo Alto, CA, 94304')
    
That is, it returns a tuple with the latitude, longitude, and address.

Use the template below to write the function. The function you write for this question should start by calling the function you wrote in Q2 to get the address. You can then use this with the API to get the location

In [67]:
def geolocate(ticker):
    import requests, re
    
    # get address for ticker
    get_address(ticker)
    # query the mapquest API for the address
    url = 'http://www.mapquestapi.com/geocoding/v1/address'
    params = {'key': API_KEY, 'location': get_address(ticker)}
    coordinates = requests.get(url, params=params).text
  

    try:
        #In  []: geolocate('TSLA')
        #Out []: (37.394738, -122.149112, '3500 Deer Creek Road, Palo Alto, CA, 94304')
        latLng = re.search(r'"latLng":{"lat":([-?0-9]+.[0-9]+),"lng":([-?0-9]+.[0-9]+)}', coordinates)
        lat = float(latLng.group(1))
        lng = float(latLng.group(2))
        
    # return the desired tuple
        return (lat, lng, get_address(ticker))

    except:
        print('Unable to find coordinates')

#for example, the code that returns the latitude, longitude, and the address of Tesla's HQ is as follows:
geolocate('TSLA')

(37.394738, -122.149112, '3500 Deer Creek RoadPalo Alto, CA 94304')

## Question 4

Now let's say we want to calculate the distance between two corporate headquarters.

Given two pairs of latitude and longitude, $(\varphi_1, \lambda_1)$ and $(\varphi_2, \lambda_2)$, we can calculate the distance between them using the [Haversine formula](https://en.wikipedia.org/wiki/Haversine_formula#The_haversine_formula):

$${\displaystyle d=2r\arcsin \left({\sqrt {\operatorname {hav} (\varphi _{2}-\varphi _{1})+\cos(\varphi _{1})\cos(\varphi _{2})\operatorname {hav} (\lambda _{2}-\lambda _{1})}}\right)}$$

where $d$ is the distance, $\operatorname{hav}$ is the Haversine function, $$\operatorname {hav} (\theta )=\left[\sin\left({\frac {\theta }{2}}\right)\right]^2,$$ and $r$ is the radius of the earth ([3958.7613 miles](https://en.wikipedia.org/wiki/Earth_radius#Mean_radius)). Note that this formula requires that the latitude and longitude are given in *radians*, not the degrees that Mapquest gives.

Use the template below to write a function that takes two ticker symbols and returns the distance between the headquarters of the two firms. 

Note that the `math` module includes the `asin` function for $\arcsin$ as well as a `radians` function.

In [68]:
def haversine(theta):
    '''Haversine function'''
    from math import sin
    return sin(theta/2)**2

In [ ]:
def firm_dist(ticker1, ticker2):
    '''Returns the distance in miles between two firm HQs'''
    
    # import whatever is needed
    from math import radians, asin, cos, sqrt
    # get lat/lng of two tickers
    lat1 = geolocate(ticker1)[0]
    lng1 = geolocate(ticker1)[1]
    lat2 = geolocate(ticker2)[0]
    lng2 = geolocate(ticker2)[1]
    
    # convert decimal degrees to radians 
    lat1, lng1, lat2, lng2 = map(radians, [lat1, lng1, lat2, lng2])
 
    # change degrees to radians
    diff_lat = lat2 - lat1 
    diff_lng = lng2 - lng1 
    
    #equation for the inside of sqrt
    i = haversine(diff_lat)+(cos(lat1)*cos(lat2)*haversine(diff_lng))
    r = 3958.7613
    d = 2 * asin(sqrt(i))*r
    
    return d
    # calculate distance using the Haversine formula and return it

#for example, the code for calculating the distance between Tesla HQ and WalMart HQ can be as follows:
firm_dist('TSLA', 'WMT')